## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt 

import requests
import json
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Atuona,-9.8000,-139.0333,81.10,scattered clouds,80,49,16.55,PF
1,1,Tasiilaq,65.6145,-37.6368,30.20,overcast clouds,93,90,1.14,GL
2,2,Mahebourg,-20.4081,57.7000,78.80,light rain,89,40,3.44,MU
3,3,Norman Wells,65.2820,-126.8329,5.00,snow,72,90,23.02,CA
4,4,Rikitea,-23.1203,-134.9692,74.55,moderate rain,85,100,12.50,PF


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Atuona,-9.8000,-139.0333,81.10,scattered clouds,80,49,16.55,PF
2,2,Mahebourg,-20.4081,57.7000,78.80,light rain,89,40,3.44,MU
5,5,Puebloviejo,10.9938,-74.2844,84.99,overcast clouds,68,100,11.99,CO
9,9,Buraydah,26.3260,43.9750,80.60,broken clouds,41,75,12.66,SA
11,11,Hobyo,5.3505,48.5268,80.78,clear sky,73,8,8.57,SO
12,12,Vaini,-21.2000,-175.2000,84.20,broken clouds,79,75,9.22,TO
13,13,Bambous Virieux,-20.3428,57.7575,78.80,light rain,89,40,3.44,MU
14,14,Riberalta,-10.9833,-66.1000,78.80,broken clouds,83,75,1.97,BO
16,16,Ternate,0.8000,127.4000,81.46,overcast clouds,80,99,5.91,ID
18,18,Phuket,8.0000,98.2500,84.99,scattered clouds,74,40,18.41,TH


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Lat                    213
Lng                    213
Max Temp               213
Current Description    213
Humidity               213
Cloudiness             213
Wind Speed             213
Country                213
dtype: int64

In [21]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Current Description    0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Atuona,-9.8000,-139.0333,81.10,scattered clouds,80,49,16.55,PF
2,2,Mahebourg,-20.4081,57.7000,78.80,light rain,89,40,3.44,MU
5,5,Puebloviejo,10.9938,-74.2844,84.99,overcast clouds,68,100,11.99,CO
9,9,Buraydah,26.3260,43.9750,80.60,broken clouds,41,75,12.66,SA
11,11,Hobyo,5.3505,48.5268,80.78,clear sky,73,8,8.57,SO
...,...,...,...,...,...,...,...,...,...,...
670,670,Moerai,-22.4333,-151.3333,80.53,clear sky,69,9,22.62,PF
675,675,Coalinga,36.1397,-120.3602,84.00,scattered clouds,17,41,3.00,US
679,679,Anloga,5.7947,0.8973,84.20,few clouds,70,20,4.61,GH
683,683,Campo Grande,-20.4428,-54.6464,77.00,few clouds,61,20,5.75,BR


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,81.10,scattered clouds,-9.8000,-139.0333,
2,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
5,Puebloviejo,CO,84.99,overcast clouds,10.9938,-74.2844,
9,Buraydah,SA,80.60,broken clouds,26.3260,43.9750,
11,Hobyo,SO,80.78,clear sky,5.3505,48.5268,
12,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,
13,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
14,Riberalta,BO,78.80,broken clouds,-10.9833,-66.1000,
16,Ternate,ID,81.46,overcast clouds,0.8000,127.4000,
18,Phuket,TH,84.99,scattered clouds,8.0000,98.2500,


In [24]:
len(hotel_df)

213

In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,81.10,scattered clouds,-9.8000,-139.0333,
2,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
5,Puebloviejo,CO,84.99,overcast clouds,10.9938,-74.2844,
9,Buraydah,SA,80.60,broken clouds,26.3260,43.9750,
11,Hobyo,SO,80.78,clear sky,5.3505,48.5268,
12,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,
13,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
14,Riberalta,BO,78.80,broken clouds,-10.9833,-66.1000,
16,Ternate,ID,81.46,overcast clouds,0.8000,127.4000,
18,Phuket,TH,84.99,scattered clouds,8.0000,98.2500,


In [26]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,81.10,scattered clouds,-9.8000,-139.0333,
2,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
5,Puebloviejo,CO,84.99,overcast clouds,10.9938,-74.2844,
9,Buraydah,SA,80.60,broken clouds,26.3260,43.9750,
11,Hobyo,SO,80.78,clear sky,5.3505,48.5268,
...,...,...,...,...,...,...,...
670,Moerai,PF,80.53,clear sky,-22.4333,-151.3333,
675,Coalinga,US,84.00,scattered clouds,36.1397,-120.3602,
679,Anloga,GH,84.20,few clouds,5.7947,0.8973,
683,Campo Grande,BR,77.00,few clouds,-20.4428,-54.6464,


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Save the figure.
plt.savefig("WeatherPy_vacation_map.png")

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>